<h1 align=center>Indo mais fundo – A mecânica do TensorFlow</h1>
<p align=center><img src=https://cdn.shortpixel.ai/spai/w_912+q_+ret_img+to_webp/https://iaexpert.academy/wp-content/uploads/2021/01/26.01.png width = 500></p>

Agora que temos alguma experiência prática com treinamento de uma Rede Neural (RN) do *TensorFlow* e aprendizado de máquina, é hora de mergulhar mais fundo na biblioteca do *TensorFlow* e explorar seu rico conjunto de recursos, o que nos permitirá implementar um aprendizado profundo mais avançado modelos nas próximas etapas.

Usaremos diferentes aspectos da API do *TensorFlow* para implementar as RNs. Em particular, usaremos novamente a API *Keras*, que fornece várias camadas de abstração para tornar a implementação de arquiteturas padrão muito conveniente. O *TensorFlow* também nos permite implementar camadas RNs personalizadas, o que é muito útil em projetos orientados à pesquisa que exigem mais personalização. Mais adiante, implementaremos essa camada personalizada.

Para ilustrar as diferentes formas de construção de modelos usando a API *Keras*, também consideraremos o problema clássico **exclusivo ou (XOR)**. Primeiramente, construiremos perceptrons multicamadas usando a classe `Sequential`. Em seguida, consideraremos outros métodos, como subclassificar `tf.keras.Model` para definir camadas personalizadas. Por fim, abordaremos o `tf.estimator`, uma API *TensorFlow* de alto nível que encapsula as etapas de aprendizado de máquina da entrada bruta à previsão.

### Como criar um gráfico no *TensorFlow* v1.x
Na versão anterior da API de baixo nível do *TensorFlow* (v1.x), esse gráfico precisava ser declarado explicitamente. As etapas individuais para criar, compilar e avaliar esse gráfico de computação no *TensorFlow* v1.x são as seguintes:
1. Instanciar um novo gráfico de computação vazio
2. Adicione nós (tensores e operações) ao gráfico de computação
3. Avalie (execute) o gráfico:
* Iniciar uma nova sessão
* Inicialize as variáveis ​​no gráfico
* Execute o gráfico de computação nesta sessão

Antes de darmos uma olhada na abordagem dinâmica no *TensorFlow* v2, vamos ver um exemplo simples que ilustra como criar um gráfico no *TensorFlow* v1.x para calcular $\small z = 2 \times (a-b) + c$. As variáveis $​​\small a$, $​​\small b$ e $​​\small c$ são escalares (números únicos) e definimos como constantes do *TensorFlow*. Um gráfico pode então ser criado chamando `tf.Graph()`. As variáveis, assim como os cálculos, representam os nós do gráfico, que definiremos da seguinte forma:

In [55]:
## TF v1.x style
import tensorflow as tf
g = tf.Graph()

with g.as_default():
    a = tf.constant(1, name='a')
    b = tf.constant(2, name='b')
    c = tf.constant(3, name='c')
    z = 2*(a-b) + c


Neste código, primeiro definimos o gráfico `g` via `g=tf.Graph()`. Em seguida, adicionamos nós ao gráfico, `g`, usando com `g.as_default()`. No entanto, observe que, se não criarmos um gráfico explicitamente, sempre haverá um gráfico padrão ao qual variáveis ​​e cálculos serão adicionados automaticamente.

No *TensorFlow* v1.x, uma sessão é um ambiente no qual as operações e os tensores de um gráfico podem ser executados. A classe `Session` foi removida do *TensorFlow* v2; No entanto, por enquanto, ele ainda está disponível por meio do submódulo `tf.compat` para permitir a compatibilidade com o *TensorFlow* v1.x. Um objeto de sessão pode ser criado chamando `tf.compat.v1.Session()`, que pode receber um gráfico existente (no caso, `g`) como argumento, como em `Session(graph=g)`.

Após lançar um gráfico em uma sessão do *TensorFlow*, podemos executar seus nós, ou seja, avaliar seus tensores ou executar seus operadores. Avaliar cada tensor individual envolve chamar seu método `eval()` dentro da sessão atual. Ao avaliar um tensor específico no gráfico, o *TensorFlow* precisa executar todos os nós anteriores no gráfico até atingir o nó de interesse fornecido. Caso haja uma ou mais variáveis *​​placeholder*, também precisamos fornecer valores para elas através do método `run` da sessão, como veremos mais adiante.

Depois de definir o gráfico estático no trecho de código anterior, podemos executar o gráfico em uma sessão do *TensorFlow* e avaliar o tensor, `z`, da seguinte forma:

In [56]:
## TF v1.x style

with tf.compat.v1.Session(graph=g) as sess:
    print('Result: z = ', sess.run(z))

Result: z =  1


### Como migrar um gráfico para o *TensorFlow* v2

Em seguida, vamos ver como esse código pode ser migrado para o *TensorFlow* v2. O *TensorFlow* v2 usa gráficos dinâmicos (em oposição aos estáticos) por padrão (isso também é chamado de execução antecipada no *TensorFlow*), o que nos permite avaliar uma operação em tempo real. Portanto, não precisamos criar explicitamente um gráfico e uma sessão, o que torna o fluxo de trabalho de desenvolvimento muito mais conveniente:

In [57]:
## TF v2 Style
a = tf.constant(1, name='a')
b = tf.constant(2, name='b')
c = tf.constant(3, name='c')

z = 2*(a - b) + c

tf.print('Result: z = ', z)



Result: z =  1


### Carregando dados de entrada em um modelo: estilo *TensorFlow* v1.x

Outra melhoria importante do *TensorFlow* v1.x para v2 diz respeito a como os dados podem ser carregados em nossos modelos. No *TensorFlow* v2, podemos alimentar dados diretamente na forma de variáveis *Python* ou arrays *NumPy*. No entanto, ao usar a API de baixo nível do *TensorFlow* v1.x, tivemos que criar variáveis de espaço reservado para fornecer dados de entrada a um modelo. Para o exemplo de gráfico de computação simples anterior, $\small z = 2 \times (a-b) + c$, vamos supor que `a`, `b` e `c` são os tensores de entrada de *Rank* 0. Podemos então definir três espaços reservados, que usaremos para "feed" dados para o modelo por meio de um dicionário *feed_dict*, da seguinte forma:

In [58]:
## TF-v1.x style
g = tf.Graph()
with g.as_default():
    a = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_a')
    b = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_b')
    c = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_c')
    z = 2*(a - b) + c
    
with tf.compat.v1.Session(graph=g) as sess:
    feed_dict = {a:1, b:2, c:3}
    print('Result: z =', sess.run(z, feed_dict=feed_dict))

Result: z = 1


### Carregando dados de entrada em um modelo: estilo *TensorFlow* v2

No *TensorFlow* v2, tudo isso pode ser feito simplesmente definindo uma função *Python* regular com `a`, `b` e `c` como seus argumentos de entrada, por exemplo:

In [59]:
## TF-v2 style
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

Agora, para realizar o cálculo, podemos simplesmente chamar essa função com objetos `Tensor` como argumentos de função. Observe que as funções do *TensorFlow*, como `add`, `subtract` e `multiply`, também nos permitem fornecer entradas de classificações mais altas na forma de um objeto *TensorFlow* `Tensor`, um array *NumPy* ou possivelmente outros objetos *Python*, como listas e tuplas. No exemplo de código a seguir, fornecemos entradas escalares (*Rank* 0), bem como entradas de *Rank* 1 e *Rank* 2, como listas:

In [60]:
tf.print('Scalar Inputs:', compute_z(1, 2, 3))
tf.print('Rank 1 Inputs:', compute_z([1], [2], [3]))
tf.print('Rank 2 Inputs:', compute_z([[1]], [[2]], [[3]]))

Scalar Inputs: 1
Rank 1 Inputs: [1]
Rank 2 Inputs: [[1]]


### Melhorando o desempenho computacional com decoradores de função

Como você viu na seção anterior, podemos escrever facilmente uma função *Python* normal e utilizar as operações do *TensorFlow*. No entanto, os cálculos por meio do modo de execução antecipada (gráfico dinâmico) não são tão eficientes quanto a execução de gráfico estático no *TensorFlow* v1.x. Assim, o *TensorFlow* v2 fornece uma ferramenta chamada **AutoGraph** que pode transformar automaticamente o código *Python* no código gráfico do *TensorFlow* para uma execução mais rápida. Além disso, o *TensorFlow* fornece um mecanismo simples para compilar uma função *Python* normal em um gráfico estático do *TensorFlow* para tornar os cálculos mais eficientes.

Para ver como isso funciona na prática, vamos trabalhar com nossa função `compute_z` anterior e anotá-la para compilação de gráfico usando o decorador `@tf.function`:

In [61]:
@tf.function
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

Observe que podemos usar e chamar essa função da mesma maneira que antes, mas agora o *TensorFlow* construirá um gráfico estático com base nos argumentos de entrada. *Python* suporta tipagem dinâmica e polimorfismo, então podemos definir uma função como `def f(a, b): return a+b` e chame-a usando entradas de *inteiro*, *float*, *list* ou *string* (lembre-se de que `a+b` é um valor válido operação para listas e *strings*). Embora os gráficos do *TensorFlow* exijam tipos e formas estáticos, o `tf.function` oferece suporte a esse recurso de digitação dinâmica. Por exemplo, vamos chamar esta função com as seguintes entradas:

In [62]:
tf.print('Scalar Inputs:', compute_z(1, 2, 3))
tf.print('Rank 1 Inputs:', compute_z([1], [2], [3]))
tf.print('Rank 2 Inputs:', compute_z([[1]], [[2]], [[3]]))

Scalar Inputs: 1
Rank 1 Inputs: [1]
Rank 2 Inputs: [[1]]


Veja que foi produzido as mesmas saídas de antes. Aqui, o *TensorFlow* usa um mecanismo de rastreamento para construir um gráfico com base nos argumentos de entrada. Para esse mecanismo de rastreamento, o *TensorFlow* gera uma tupla de chaves com base nas assinaturas de entrada fornecidas para chamar a função. As chaves geradas são as seguintes:

* Para argumentos `tf.Tensor`, a chave é baseada em seus *shapes* e *dtypes*.
* Para tipos *Python*, como listas, seu `id()` é usado para gerar chaves de cache.
* Para valores primitivos do *Python*, as chaves de cache são baseadas nos valores de entrada.

Ao chamar essa função decorada, o *TensorFlow* verificará se um gráfico com a chave correspondente já foi gerado. Se esse gráfico não existir, o *TensorFlow* gerará um novo gráfico e armazenará a nova chave. Por outro lado, se quisermos limitar a forma como uma função pode ser chamada, podemos especificar sua assinatura de entrada por meio de uma tupla de objetos `tf.TensorSpec` ao definir a função. Por exemplo, vamos redefinir a função anterior, `compute_z`, e especificar que apenas tensores de rank 1 do tipo `tf.int32` são permitidos:

In [63]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),
                              tf.TensorSpec(shape=[None], dtype=tf.int32),
                              tf.TensorSpec(shape=[None], dtype=tf.int32),))
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

Agora, podemos chamar esta função usando tensores de rank 1 (ou listas que podem ser convertidas em tensores de rank 1):

In [64]:
tf.print('Rank 1 Inputs:', compute_z([1], [2], [3]))
tf.print('Rank 1 Inputs:', compute_z([1, 2], [2, 4], [3, 6]))

Rank 1 Inputs: [1]
Rank 1 Inputs: [1 2]


No entanto, chamar essa função usando tensores com classificações diferentes de 1 resultará em um erro, pois a classificação não corresponderá à assinatura de entrada especificada, conforme a seguir:

In [65]:
tf.print('Rank 0 Inputs:', compute_z(1, 2, 3))

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    1,
    2,
    3)
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None),
    TensorSpec(shape=(None,), dtype=tf.int32, name=None),
    TensorSpec(shape=(None,), dtype=tf.int32, name=None)).

In [66]:
tf.print('Rank 2 Inputs:', compute_z([[1], [2]], [[2], [4]], [[3], [6]]))

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    [[1], [2]],
    [[2], [4]],
    [[3], [6]])
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None),
    TensorSpec(shape=(None,), dtype=tf.int32, name=None),
    TensorSpec(shape=(None,), dtype=tf.int32, name=None)).

### Objetos variáveis do *TensorFlow* para armazenar e atualizar os parâmetros do modelo

No contexto do *TensorFlow*, uma `Variable` é um objeto Tensor especial que nos permite armazenar e atualizar os parâmetros de nossos modelos durante o treinamento. Um `Variable` pode ser criada apenas chamando a classe `tf.Variable` em valores iniciais especificados pelo usuário. No código a seguir, vamos gerar objetos `Variable` do tipo `float32`, `int32`, `bool` e `string`:

In [67]:
a = tf.Variable(initial_value=3.14, name='var_a')
b = tf.Variable(initial_value=[1, 2, 3], name='var_b')
c = tf.Variable(initial_value=[True, False], dtype=tf.bool)
d = tf.Variable(initial_value=['abc'], dtype=tf.string)
print(a)
print(b)
print(c)
print(d)

<tf.Variable 'var_a:0' shape=() dtype=float32, numpy=3.14>
<tf.Variable 'var_b:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3])>
<tf.Variable 'Variable:0' shape=(2,) dtype=bool, numpy=array([ True, False])>
<tf.Variable 'Variable:0' shape=(1,) dtype=string, numpy=array([b'abc'], dtype=object)>


Observe que sempre temos que fornecer os valores iniciais ao criar um `Variable`. As variáveis têm um atributo chamado `treinable`, que, por padrão, é definido como `True`. APIs de nível superior, como *Keras*, usarão esse atributo para gerenciar as variáveis treináveis e não treináveis. Você pode definir um `Variable` não treinável da seguinte forma:

In [68]:
w = tf.Variable([1,2,3], trainable=False)

print(w.trainable)

False


Os valores de um `Variable` podem ser modificados de forma eficiente executando algumas operações como `.assign()`, `.assign_add()` e métodos relacionados. Vejamos alguns exemplos:

In [69]:
print(w.assign([3,1,4], read_value=True))

<tf.Variable 'UnreadVariable' shape=(3,) dtype=int32, numpy=array([3, 1, 4])>


In [70]:
w.assign_add([2,-1,2], read_value=False)

print(w.value())

tf.Tensor([5 0 6], shape=(3,), dtype=int32)


Quando o argumento `read_value` estiver definido como `True` (que também é o padrão), essas operações retornarão automaticamente os novos valores após atualizar os valores atuais da `Variable`. Definir o `read_value` como `False` suprimirá o retorno automático do valor atualizado (mas a `Variable` ainda será atualizada no local). Chamar `w.value()` retornará os valores em um formato de tensor. Observe que não podemos alterar a forma ou o tipo da `Variable` durante a atribuição.

Você deve se lembrar de que, para modelos de Rede Neural, é necessário inicializar os parâmetros do modelo com pesos aleatórios para quebrar a simetria durante a retropropagação - caso contrário, uma RN multicamada não seria mais útil do que uma RN de camada única, como regressão logística. Ao criar uma variável do *TensorFlow*, também podemos usar um esquema de inicialização aleatório. O *TensorFlow* pode gerar números aleatórios com base em várias distribuições via `tf.random`.

No exemplo a seguir, veremos alguns métodos de inicialização padrão que também estão disponíveis no *Keras*. Então, vamos ver como podemos criar uma `Variable` com inicialização *Glorot*, que é um esquema clássico de inicialização aleatória que foi proposto por Xavier Glorot e Yoshua Bengio. Para isso, criamos um operador chamado `init` como objeto da classe `GlorotNormal`. Então, chamamos esse operador e fornecemos a forma desejada do tensor de saída:

In [71]:
tf.random.set_seed(1)
init = tf.keras.initializers.GlorotNormal()

tf.print(init(shape=(3,)))

[-0.722795904 1.01456821 0.251808226]


Agora, podemos usar este operador para inicializar uma `Variable` de formato 2 × 3 :

In [72]:
v = tf.Variable(init(shape=(2, 3)))
tf.print(v)

[[0.28982234 -0.782292783 -0.0453658961]
 [0.960991383 -0.120003454 0.708528221]]


Agora, para colocar isso no contexto de um caso de uso mais prático, vamos ver como podemos definir uma `Variable` dentro da classe base `tf.Module`. Vamos definir duas variáveis: uma treinável e uma não treinável:

In [73]:
class MyModule (tf.Module):
    def __init__(self):
        init = tf.keras.initializers.GlorotNormal()
        self.w1 = tf.Variable(init(shape=(2,3)),
                                    trainable=True)
        self.w2 = tf.Variable(init(shape=(1,2)),
                                trainable = False)

m = MyModule()

print('All module variables:', [v.shape for v in m.variables])

print('Trainable variable:  ', [v.shape for v in m.trainable_variables])



All module variables: [TensorShape([2, 3]), TensorShape([1, 2])]
Trainable variable:   [TensorShape([2, 3])]


Como você pode ver neste exemplo de código, a subclasse da classe `tf.Module` nos dá acesso direto a todas as variáveis definidas em um determinado objeto (aqui, uma instância de nossa classe `MyModule` personalizada) por meio do atributo `.variables`.

Finalmente, vamos ver como usar variáveis dentro de uma função decorada com `tf.function`. Quando definimos uma `Variable` do TensorFlow dentro de uma função normal (não decorada), podemos esperar que uma nova `Variable` seja criada e inicializada toda vez que a função for chamada. No entanto, `tf.function` tentará reutilizar a `Variable` com base no rastreamento e na criação do gráfico. Portanto, o *TensorFlow* não permite a criação de uma `Variable` dentro de uma função decorada e, como resultado, o código a seguir gerará um erro:

In [74]:
@tf.function
def f(x):
    w = tf.Variable([1,2,3])

f([1])

ValueError: in user code:

    File "C:\Users\willi\AppData\Local\Temp/ipykernel_11580/3931184721.py", line 3, in f  *
        w = tf.Variable([1,2,3])

    ValueError: tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.


Uma maneira de evitar esse problema é definir a variável fora da função decorada e usá-la dentro da função:

In [75]:
w = tf.Variable(tf.random.uniform((3, 3)))

@ tf.function
def compute_z(x):
    return tf.matmul(w , x)

x = tf.constant([[1], [2], [3]], dtype=tf.float32)

tf.print(compute_z(x))

[[1.20935762]
 [3.89828062]
 [1.65398622]]


### Computação de gradientes via diferenciação automática e *GradientTape*

Como você já sabe, otimizar RNs requer calcular os gradientes do custo em relação aos pesos da Rn. Isso é necessário para algoritmos de otimização, como gradiente descendente estocástico (SGD). Além disso, os gradientes têm outras aplicações, como diagnosticar a rede para descobrir por que um modelo de RN está fazendo uma previsão específica para um exemplo de teste. Portanto, nesta seção, abordaremos como calcular gradientes de uma computação em relação a algumas variáveis.

### Calculando os gradientes da perda em relação às variáveis ​​treináveis

O *TensorFlow* oferece suporte à **diferenciação automática**, que pode ser considerada uma implementação da regra da cadeia para calcular gradientes de funções aninhadas. Quando definimos uma série de operações que resultam em alguma saída ou mesmo tensores intermediários, o *TensorFlow* fornece um contexto para calcular gradientes desses tensores calculados em relação a seus nós dependentes no gráfico de computação. Para calcular esses gradientes, temos que "gravar" os cálculos via `tf.GradientTape`.

Vamos trabalhar com um exemplo simples onde vamos calcular $\small z = wx + b$ e definir a perda como a perda ao quadrado entre o alvo e a previsão, $\small Loss = (y - z)^2$. No caso mais geral, onde podemos ter várias previsões e metas, calcule a perda como a soma do erro ao quadrado, $\small Loss = \sum_i(y_i - z_i)^2$. Para implementar esse cálculo no *TensorFlow*, vamos definir os parâmetros do modelo, $\small w$ e $\small b$, como variáveis, e as entradas $\small x$ e $\small y$, como tensores. Colocaremos o cálculo de $\small z$ e a perda dentro do contexto `tf.GradientTape`:

In [100]:
import tensorflow as tf

w = tf.Variable(1.0)
b = tf.Variable(0.5)
print(w.trainable, b.trainable)

True True


In [101]:
x = tf.convert_to_tensor([1.4])
y = tf.convert_to_tensor([2.1])

with tf.GradientTape() as tape:
    z = tf.add(tf.multiply(w, x), b)
    loss = tf.reduce_sum(tf.square(y - z))

dloss_dw = tape.gradient(loss, w)

tf.print('dL/dw : ', dloss_dw)

dL/dw :  -0.559999764


Ao calcular o valor $\small z$, podemos pensar nas operações necessárias, que gravamos na "gradient tape", como uma passagem para frente em uma RN. Usamos `tape.gradient` para calcular $\small \dfrac{\partial Loss}{\partial w}$ . Como este é um exemplo muito simples, podemos obter as derivadas, $\small \dfrac {\partial Loss}{\partial w} = 2x(wx + b - y)$, simbolicamente para verificar que os gradientes calculados obtivemos no exemplo de código anterior:

In [78]:
# verifying the computed gradient
tf.print(2*x*(w*x  + b - y))

[-0.559999764]


### Computação de gradientes em relação a tensores não treináveis

`tf.GradientTape` suporta automaticamente os gradientes para variáveis treináveis. No entanto, para variáveis não treináveis e outros objetos `Tensor`, precisamos adicionar uma modificação adicional ao `GradientTape` chamada `tape.watch()` para monitorá-los também. Por exemplo, se estivermos interessados em calcular $\small \dfrac {\partial Loss}{\partial x}$ , o código será o seguinte:

In [106]:
with tf.GradientTape() as tape:
    tape.watch(x)
    z = tf.add(tf.multiply(w, x), b)
    loss = tf.square(y - z)

dloss_dx = tape.gradient(loss, x)

tf.print('dL/dx:', dloss_dx)

dL/dx: [-0.399999857]


In [107]:
# verifying the computed gradient
tf.print(2*w * ((w*x + b) - y))

[-0.399999857]


### Mantendo recursos para vários cálculos de gradiente
Quando monitoramos os cálculos no contexto de`tf.GradientTape`, por padrão, a fita manterá os recursos apenas para um único cálculo de gradiente. Por exemplo, após chamar `tape.gradient()` uma vez, os recursos serão liberados e a fita será limpa. Portanto, se quisermos calcular mais de um gradiente, por exemplo, ambos $\small \dfrac {\partial Loss}{\partial w}$ e $\small \dfrac {\partial Loss}{\partial b}$ , precisamos tornar a fita persistente:

In [108]:
with tf.GradientTape(persistent=True) as tape:
    z = tf.add(tf.multiply(w, x), b)
    loss = tf.reduce_sum(tf.square(y - z))

dloss_dw = tape.gradient(loss, w)
dloss_db = tape.gradient(loss, b)

tf.print('dL/dw:', dloss_dw)
tf.print('dL/db:', dloss_db)

dL/dw: -0.559999764
dL/db: -0.399999857


In [105]:
tf.print(2*x * ((w*x + b) - y))

[-0.559999764]


No entanto, lembre-se de que isso só é necessário quando queremos calcular mais de um gradiente, pois gravar e manter a fita gradiente é menos eficiente em termos de memória em comparação com liberar a memória após um único cálculo de gradiente. É também por isso que a configuração padrão é `persistent=False`.

Finalmente, se estivermos computando gradientes de um termo de perda em relação aos parâmetros de um modelo, podemos definir um otimizador e aplicar os gradientes para otimizar os parâmetros do modelo usando a API `tf.keras`, como segue:

In [109]:
optimizer = tf.keras.optimizers.SGD()

optimizer.apply_gradients(zip([dloss_dw, dloss_db], [w, b]))

tf.print('Updated w:', w)
tf.print('Updated bias:', b)

AttributeError: module 'tensorflow.compat.v2.__internal__.distribute' has no attribute 'strategy_supports_no_merge_call'

In [ ]:
b

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.5>